<a href="https://colab.research.google.com/github/sharunsha/sharunsha/blob/main/Option%20Chain%20Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Option Chain Data Fetch and Margin/Premium Calculation

# Project Overview
This project provides a Python-based solution to retrieve options data for a specified financial instrument, filter and process it, and calculate key financial metrics, such as margin requirements and premiums earned on options trades. This code can be adapted for use with any broker’s API that provides an options chain data feed and margin calculation endpoint.

### Code Structure and Logic

# Fetching and Filtering Option Chain Data

Function: get_option_chain_data

Purpose: Retrieve option chain data for a given instrument (e.g., NIFTY) and filter based on the option type (e.g., Call or Put).

**Parameters:**

1. Instrument_name (str): Name of the instrument, e.g., "NIFTY."
2. Expiry_date (str): The options contract's expiry date.
3. Side (str): Type of option to filter, either "PE" (Put) or "CE" (Call).

**Logic:**


1. An API call is made to fetch options data for the specified instrument and expiry.
2. The response is processed to filter data based on the option side (PE or CE) and selects either the bid price (for PE) or ask price (for CE).
3.  The filtered options data is then returned in a pandas DataFrame.







In [8]:
import pandas as pd
import requests  # Used for API calls


In [9]:
def get_option_chain_data(instrument_name: str, expiry_date: str, side: str) -> pd.DataFrame:
    # Define API endpoint and headers (example structure; replace with actual details)
    api_url = "https://api.broker.com/options_chain"  # Replace with the actual broker's API URL
    headers = {
        "Authorization": "Bearer YOUR_API_KEY",  # Replace with your actual API key
    }
    params = {
        "symbol": instrument_name,
        "expiry_date": expiry_date,
    }

    # Fetch data from the API
    response = requests.get(api_url, headers=headers, params=params)
    data = response.json()  # Assuming the API returns JSON

    # Check if data retrieval was successful
    if response.status_code != 200:
        print("Failed to fetch data")
        return pd.DataFrame()  # Return an empty DataFrame on failure

    # Process data to filter and find highest bid/ask price based on the option type (PE or CE)
    filtered_data = []
    for option in data['options']:  # Assuming 'options' contains options data
        if option['type'] == side:
            highest_price = option['bid_price'] if side == "PE" else option['ask_price']
            filtered_data.append({
                "instrument_name": instrument_name,
                "strike_price": option['strike_price'],
                "side": side,
                "bid/ask": highest_price
            })

    # Convert the filtered data to a DataFrame
    df = pd.DataFrame(filtered_data)
    return df


In [10]:
# Mock function to simulate API data for testing
def mock_get_option_chain_data(instrument_name: str, expiry_date: str, side: str) -> pd.DataFrame:
    # Simulated API data structure
    mock_data = {
        "options": [
            {"strike_price": 19500, "type": "CE", "ask_price": 2302.25, "bid_price": 0.65},
            {"strike_price": 19500, "type": "PE", "ask_price": 0.55, "bid_price": 0.75},
            {"strike_price": 19600, "type": "CE", "ask_price": 2200.50, "bid_price": 1.05},
            {"strike_price": 19600, "type": "PE", "ask_price": 1.00, "bid_price": 1.25},
        ]
    }

    # Filter data as per the real function's logic
    filtered_data = []
    for option in mock_data['options']:
        if option['type'] == side:
            highest_price = option['bid_price'] if side == "PE" else option['ask_price']
            filtered_data.append({
                "instrument_name": instrument_name,
                "strike_price": option['strike_price'],
                "side": side,
                "bid/ask": highest_price
            })

    # Convert to DataFrame
    df = pd.DataFrame(filtered_data)
    return df

# Test the mock function
test_df = mock_get_option_chain_data("NIFTY", "2024-12-31", "CE")
print(test_df)


  instrument_name  strike_price side  bid/ask
0           NIFTY         19500   CE  2302.25
1           NIFTY         19600   CE  2200.50


# Calculating Margin Requirement and Premium Earned
Function: calculate_margin_and_premium

Purpose:
Calculate the margin requirement and premium earned on each option.
**Parameters:**

1. data (pd.DataFrame): DataFrame containing option chain data returned by get_option_chain_data.
lot_size (int): The number of shares per option contract (e.g., 75).
**Logic:**
1. For each option, an API call is made to calculate the margin based on the transaction type "SELL."
2. The premium earned is calculated by multiplying the option’s bid/ask price by the lot size.
3. The function then returns a modified DataFrame with two new columns: margin_required and premium_earned.

Testing: A mock version of calculate_margin_and_premium is provided to simulate margin calculations, which can be replaced with actual API integration.

In [11]:
def calculate_margin_and_premium(data: pd.DataFrame, lot_size: int = 75) -> pd.DataFrame:
    # API endpoint for margin calculation (replace with actual URL)
    margin_api_url = "https://api.broker.com/calculate_margin"

    # Add new columns to store margin and premium values
    data["margin_required"] = 0.0
    data["premium_earned"] = 0.0

    for index, row in data.iterrows():
        # Example margin API request (replace with actual API call details)
        margin_params = {
            "symbol": row["instrument_name"],
            "strike_price": row["strike_price"],
            "side": row["side"],
            "transaction_type": "SELL"
        }

        # Assume margin calculation API returns margin requirement directly
        response = requests.get(margin_api_url, params=margin_params)
        if response.status_code == 200:
            margin_data = response.json()
            data.at[index, "margin_required"] = margin_data.get("margin_required", 0.0)
        else:
            print(f"Failed to retrieve margin for {row['instrument_name']} {row['strike_price']} {row['side']}")

        # Calculate premium earned (bid/ask price * lot size)
        data.at[index, "premium_earned"] = row["bid/ask"] * lot_size

    return data


In [12]:
def calculate_margin_and_premium(data: pd.DataFrame, lot_size: int = 75) -> pd.DataFrame:
    # Remove API call for testing - Use mock margin data instead

    # Add new columns to store margin and premium values
    data["margin_required"] = 0.0
    data["premium_earned"] = 0.0

    for index, row in data.iterrows():
        # Mock margin requirement for testing purposes
        data.at[index, "margin_required"] = 2000.0  # Replace with actual margin logic when available

        # Calculate premium earned (bid/ask price * lot size)
        data.at[index, "premium_earned"] = row["bid/ask"] * lot_size

    return data


In [13]:
import pandas as pd

# Function to get option chain data (using mock data for testing)
def get_option_chain_data(instrument_name: str, expiry_date: str, side: str) -> pd.DataFrame:
    # Mock data for option chain
    data = {
        "instrument_name": [instrument_name, instrument_name],
        "strike_price": [19500, 19500],
        "side": ["PE", "CE"],
        "bid/ask": [0.65, 2302.25]  # Mock bid price for PE and ask price for CE
    }
    return pd.DataFrame(data)

# Function to calculate margin and premium (mock margin data for testing)
def calculate_margin_and_premium(data: pd.DataFrame, lot_size: int = 75) -> pd.DataFrame:
    # Add columns for margin and premium values
    data["margin_required"] = 0.0
    data["premium_earned"] = 0.0

    for index, row in data.iterrows():
        # Using mock margin value for testing
        data.at[index, "margin_required"] = 2000.0  # Replace with API call if available

        # Calculate premium earned
        data.at[index, "premium_earned"] = row["bid/ask"] * lot_size

    return data

# Step-by-step process to test the functions

# 1. Fetch option chain data
options_data = get_option_chain_data("NIFTY", "2024-12-28", "PE")
print("Option Chain Data:")
print(options_data)

# 2. Calculate margin and premium
result_data = calculate_margin_and_premium(options_data)
print("\nFinal Data with Margin and Premium Calculated:")
print(result_data)


Option Chain Data:
  instrument_name  strike_price side  bid/ask
0           NIFTY         19500   PE     0.65
1           NIFTY         19500   CE  2302.25

Final Data with Margin and Premium Calculated:
  instrument_name  strike_price side  bid/ask  margin_required  premium_earned
0           NIFTY         19500   PE     0.65           2000.0           48.75
1           NIFTY         19500   CE  2302.25           2000.0       172668.75


# Approach and Specific Details
The approach used here involves modular, reusable functions designed for flexibility. The code allows for easy integration with various brokers’ APIs by modifying the endpoint URLs and authorization details. The use of pandas DataFrames provides a clean, tabular format for data manipulation, ensuring easy data processing and visualization.

# **AI Tools Used**
OpenAI's GPT-3.5 (ChatGPT) was used in the development process to assist in:

**Code Structuring**: Designing modular functions and clear parameter definitions.

**Error Handling**: Generating error-handling patterns to manage common issues such as connection failures.

**Documentation**: Providing structured guidance for this README file

**Requirements**: Ensure that you have the following Python packages installed:

**Pandas**: For data manipulation.

**requests: For making HTTP requests to the API.**

In [14]:
pip install pandas requests